# Notion Ops (Work Token) via ultimate-notion (UNO)

Status: WIP
Owner: Hugo Evers
GitHub issue: https://github.com/hugocool/FateForger/issues/15
Issue branch: issue/14-tasks-agent-ticktick-lists
PR: TBD
notebook-mode: notebook-mode
Last clean run: 2026-02-16 (.venv, Python 3.11.9)
Repo cleanliness snapshot: dirty (32 entries in `git status --porcelain`) @ 2026-02-16 23:57 +0100

Targets (work Notion)
- Tasks DBs: `110baca1-857f-48b1-a8ec-cea325202eef`, `d0a4103c-d10a-4860-9a71-6bf4fc55192f`
- Projects DBs: `c37cbc40-020e-40db-88eb-d460e266b08b`, `9f34f42d-d3d4-449d-8d39-ee7d529fbfd4`

References
- ultimate-notion docs: https://ultimate-notion.com/
- Notion API reference: https://developers.notion.com/reference/intro


## Pairing Intake Record

Problem
- Two Notion tenants (personal + work). We want to operate on the work tenant using `WORK_NOTION_TOKEN` from `.env`.
- Tasks already have execution `Status` (Not started / In progress / Done / Archived).
- DoD is handled either in-page or via existing properties; artifacts are linked via the Resources relation.
- Work type should be derived from the Project a task is associated with.
- Add a simple refinement/triage dimension ("Ticket Status") without rewriting the execution status.

Constraints
- Non-destructive: no deletes, no renames, no rewriting existing statuses.
- Default dry-run: do not mutate Notion unless explicitly enabled.
- Avoid printing secrets/tokens.


## Design Options

Option A: Use `ultimate-notion` (UNO) for schema + page writes (recommended)
- Pros: typed schema objects, safer ergonomics than raw JSON.
- Cons: some Notion API features may not be supported; must follow UNO config conventions.

Option B: Use raw Notion API via `httpx`
- Pros: maximum control.
- Cons: more boilerplate, easier to make destructive mistakes.

Selected
- Option A.


## Implementation Walkthrough / Decision Audit

Planned changes (minimal)
- Projects DBs: add `Work Type` (select) with options: Revenue, Product, Research, System, Visibility.
- Tasks DBs: add `Ticket Status` (select) with options: Unrefined, Refined, Ready, Blocked.
- Tasks DBs: add `Work Type` rollup (Project -> Work Type), calculate: show original.
- Projects DBs: optionally create missing project pages: Finances, Marketing, Sales, BizDev.

Non-goals
- No TickTick sync here.
- No view creation via API (manual steps only).


In [ ]:
from __future__ import annotations

import os
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable

import ultimate_notion as uno
from ultimate_notion import Session, schema

from fateforger.setup_wizard.envfile import read_env_file


# Toggle: keep False unless you want to write to Notion.
APPLY_CHANGES = False

# Work-tenant token is stored separately to keep personal/work split explicit.
ENV = read_env_file(Path(".env"))
WORK_TOKEN = (ENV.get("WORK_NOTION_TOKEN") or "").strip()
if not WORK_TOKEN:
    raise RuntimeError("WORK_NOTION_TOKEN missing in .env")

# ultimate-notion reads token from its config/env; it ignores Session(auth=...).
# Hacky but effective: set env var before creating the Session, and force a new active session.
os.environ["NOTION_TOKEN"] = WORK_TOKEN
Session._active_session = None  # type: ignore[attr-defined]

TASKS_DB_IDS = [
    "110baca1-857f-48b1-a8ec-cea325202eef",
    "d0a4103c-d10a-4860-9a71-6bf4fc55192f",
]
PROJECTS_DB_IDS = [
    "c37cbc40-020e-40db-88eb-d460e266b08b",
    "9f34f42d-d3d4-449d-8d39-ee7d529fbfd4",
]

WORK_TYPE_OPTIONS = ["Revenue", "Product", "Research", "System", "Visibility"]
TICKET_STATUS_OPTIONS = ["Unrefined", "Refined", "Ready", "Blocked"]
NEW_PROJECT_PAGES = ["Finances", "Marketing", "Sales", "BizDev"]


@dataclass(frozen=True)
class PlannedChange:
    target: str
    action: str
    details: str


def _print_plan(planned: list[PlannedChange]) -> None:
    if not planned:
        print("No changes needed.")
        return
    print("Planned changes:")
    for ch in planned:
        print(f"- [{ch.target}] {ch.action}: {ch.details}")


In [ ]:
notion = Session.get_or_create()

tasks_dbs = [notion.get_db(db_id, use_cache=False) for db_id in TASKS_DB_IDS]
projects_dbs = [notion.get_db(db_id, use_cache=False) for db_id in PROJECTS_DB_IDS]

print("Loaded DBs:")
for db in tasks_dbs:
    print(f"- Tasks: {db.title} ({db.id})")
for db in projects_dbs:
    print(f"- Projects: {db.title} ({db.id})")

print("\nTasks schemas:")
for db in tasks_dbs:
    print("\n---")
    print(db.schema)

print("\nProjects schemas:")
for db in projects_dbs:
    print("\n---")
    print(db.schema)


In [ ]:
def ensure_select_property(
    db: uno.database.Database,
    *,
    prop_name: str,
    options: Iterable[str],
    planned: list[PlannedChange],
) -> None:
    if db.schema.has_prop(prop_name):
        return
    planned.append(
        PlannedChange(
            target=str(db.id),
            action="add_property",
            details=f"{prop_name} (select) options={list(options)}",
        )
    )
    if not APPLY_CHANGES:
        return
    db.schema[prop_name] = schema.Select(options=[schema.Option(o) for o in options])


def ensure_rollup_work_type(
    *,
    tasks_db: uno.database.Database,
    projects_db: uno.database.Database,
    planned: list[PlannedChange],
) -> None:
    rollup_name = "Work Type"
    if tasks_db.schema.has_prop(rollup_name):
        return
    if not tasks_db.schema.has_prop("Project"):
        planned.append(
            PlannedChange(
                target=str(tasks_db.id),
                action="skip_rollup",
                details="No Project relation found on tasks DB",
            )
        )
        return
    if not projects_db.schema.has_prop("Work Type"):
        planned.append(
            PlannedChange(
                target=str(tasks_db.id),
                action="skip_rollup",
                details="Projects DB missing Work Type; add it first",
            )
        )
        return

    planned.append(
        PlannedChange(
            target=str(tasks_db.id),
            action="add_property",
            details="Work Type (rollup: Project -> Work Type)",
        )
    )
    if not APPLY_CHANGES:
        return
    tasks_db.schema[rollup_name] = schema.Rollup(
        relation=tasks_db.schema["Project"],
        rollup=projects_db.schema["Work Type"],
    )


def ensure_project_pages(
    projects_db: uno.database.Database,
    *,
    names: Iterable[str],
    planned: list[PlannedChange],
) -> None:
    existing = {p.title.strip().lower() for p in projects_db.get_all_pages()}
    title_attr = projects_db.schema.get_title_prop().attr_name

    for name in names:
        if name.strip().lower() in existing:
            continue
        planned.append(
            PlannedChange(
                target=str(projects_db.id),
                action="create_page",
                details=f"Project: {name}",
            )
        )
        if not APPLY_CHANGES:
            continue
        projects_db.create_page(**{title_attr: name})


planned: list[PlannedChange] = []

# 1) Projects DBs: Work Type + missing project pages.
for pdb in projects_dbs:
    ensure_select_property(
        pdb,
        prop_name="Work Type",
        options=WORK_TYPE_OPTIONS,
        planned=planned,
    )
    ensure_project_pages(pdb, names=NEW_PROJECT_PAGES, planned=planned)

# 2) Tasks DBs: Ticket Status + Work Type rollup.
for tdb, pdb in zip(tasks_dbs, projects_dbs, strict=False):
    ensure_select_property(
        tdb,
        prop_name="Ticket Status",
        options=TICKET_STATUS_OPTIONS,
        planned=planned,
    )
    ensure_rollup_work_type(tasks_db=tdb, projects_db=pdb, planned=planned)

_print_plan(planned)
print("\nAPPLY_CHANGES=", APPLY_CHANGES)


## Reviewer Checklist

- Run the notebook with `APPLY_CHANGES=False` and confirm the planned changes look correct.
- Flip `APPLY_CHANGES=True`, rerun, and verify in Notion UI:
  - Projects DBs have `Work Type` property.
  - Tasks DBs have `Ticket Status` property.
  - Tasks DBs have `Work Type` rollup.
  - New project pages exist if they were missing.


## Open Items

To decide
- Whether to add more `Ticket Status` options (e.g., Zombie) or keep it minimal.
- Whether to backfill `Work Type` on existing project pages immediately.

To do
- Add manual Notion views: Zombie view + Committed cap view.

Blocked by
- none


## Acceptance Criteria Checklist

- [ ] Notebook scaffold exists and maps to GitHub issue.
- [ ] Dry-run produces a deterministic plan and makes no writes.
- [ ] Apply mode adds only properties/pages (non-destructive).
- [ ] Token safety: no tokens printed.


## Extraction Map (Notebook -> Artifacts)

- Notebook-only by design for now.


## Closeout / Remaining Notebook-Only Content

- This notebook is intended to remain a runbook until multi-tenant support is implemented properly.
